# Set up

In [30]:
!git clone https://github.com/NLP-Reichman/2025_assignment_1.git
!mv 2025_assignment_1/data data
!rm 2025_assignment_1/ -r

1375.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
fatal: destination path '2025_assignment_1' already exists and is not an empty directory.
1380.58s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
mv: 2025_assignment_1/data: No such file or directory
1385.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
rm: 2025_assignment_1/: is a directory
rm: -r: No such file or directory


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [1]:
import os
import json
# from google.colab import files

import pandas as pd
import numpy as np
from itertools import product

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [2]:
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  vocab = []
  ########################################
  
  vocab_set = set()
  for file in os.listdir('data'):
    if file.endswith('.json'):
      with open(os.path.join('data', file), 'r') as f:
        data = pd.read_json(f)
        text = data["tweet_text"]
        for tweet in text:
          for char in tweet:
            if char not in vocab_set:
              vocab_set.add(char)
              vocab.append(char)
  vocab.append("<start>")
  vocab.append("<end>")

  ########################################
  return vocab

In [3]:
vocab = preprocess()
print(f"vocab length: {len(vocab)}")
print(f"Some characters in the vocab: {vocab[:10]}")

vocab length: 1804
Some characters in the vocab: ['R', 'T', ' ', '@', 'V', 'O', '_', 'N', 'L', ':']


## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1|V|$ to the LM if building a smoothed LM.

In [15]:
def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
  '''
  Return a language model for the given lang and n_gram (n)
  :param lang: the language of the model
  :param n: the n_gram value
  :param smoothed: boolean indicating whether to apply smoothing
  :return: a dictionary where the keys are n_grams and the values are dictionaries
  '''
  LM = {}
  ########################################
  tweets = []
  for file in os.listdir('data'):
    if file.startswith(lang) and file.endswith('.json'):
      with open(os.path.join('data', file), 'r') as f:
        data = pd.read_json(f)
        tweets = data["tweet_text"]
        break
  
  # Initialize the language model
  for tweet in tweets:
    # Add start tokens and end token
    processed_tweet = ["<start>"] * (n-1) + list(tweet) + ["<end>"]
    
    for i in range(len(processed_tweet) - n + 1):
      prefix = ''.join(processed_tweet[i:i+n-1])
      next_char = processed_tweet[i+n-1]
      
      if prefix not in LM:
        LM[prefix] = {}
      
      if next_char not in LM[prefix]:
        LM[prefix][next_char] = 0

      LM[prefix][next_char] += 1
  
  # Apply add-one smoothing if required
  if smoothed:
    vocab = preprocess()
    vocab_size = len(vocab)
    
    for prefix in LM:
      total_count = sum(LM[prefix].values())
      for next_char in LM[prefix]:
        LM[prefix][next_char] = (LM[prefix][next_char] + 1) / (total_count + vocab_size)
      
      # Add the unknown token
      LM[prefix]["<unk>"] = 1 / (total_count + vocab_size)
  else:
    # Convert counts to probabilities without smoothing
    for prefix in LM:
      total_count = sum(LM[prefix].values())
      for next_char in LM[prefix]:
        LM[prefix][next_char] = LM[prefix][next_char] / total_count
        
  ########################################
  return LM

In [5]:
LM = build_lm("en", 3, False)
print(f"English Language Model with 3-gram is of length: {len(LM)}")

English Language Model with 3-gram is of length: 8238


## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [16]:
def perplexity(model: dict, text: list, n) -> float:
  '''
  Calculates the perplexity of the given string using the given language model.
  :param model: The language model
  :param text: The tokenized text to calculate the perplexity for
  :param n: The n-gram of the model
  :return: The perplexity
  '''
  pp = 0
  # Add start and end tokens
  processed_text = ["<start>"] * (n-1) + list(text) + ["<end>"]
  
  N = len(processed_text)
  log_sum = 0
  for i in range(n-1, N):
    if n == 1:
      context = ""
    else:
      context = ''.join(processed_text[i-(n-1):i])
    char = processed_text[i]
    
    if context in model:
      # check this
      prob = model[context].get(char, model[context].get("<unk>", 1/len(model[context])))
    else:
      prob = 1/len(model)
    log_sum += np.log2(prob)
  
  # Calculate perplexity
  pp = 2 ** (-1/(N - (n-1)) * log_sum)
  return pp

In [17]:
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :param n: The n-gram of the model
  :return: the perplexity value
  '''
  pp = 0
  ########################################
  # Read the test file for the target language
  text = []
  for file in os.listdir('data'):
    if file == target_lang + '.json':
      with open(os.path.join('data', file), 'r') as f:
        data = pd.read_json(f)
        text = data["tweet_text"]
        break
  # Calculate perplexity using the provided model and n-gram value  
  for tweet in text:
    pp += perplexity(model, tweet, n)
  pp /= len(text)
  ########################################
  return pp


In [18]:
LM = build_lm("en", 3, True)

In [19]:
print("Perplexity of the English 3-gram model on datasets:")
print(f"On English: {eval(LM, 'en', 3): .2f}")
print(f"On French: {eval(LM, 'fr', 3): .2f}")
print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")


Perplexity of the English 3-gram model on datasets:
On English:  31.62


TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable log2 method

In [10]:
lm1 = build_lm("en", 1, True)
lm2 = build_lm("en", 2, True)
lm3 = build_lm("en", 3, True)
lm4 = build_lm("en", 4, True)

print("Perplexity on different n-gram models on English")
print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

Perplexity on different n-gram models on English
On 1-gram:  42.73
On 2-gram:  23.00
On 3-gram:  31.62
On 4-gram:  73.45


## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [11]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [12]:
def match() -> pd.DataFrame:
    df = pd.DataFrame(columns=['source_lang', 'target_lang', 'n', 'perplexity'])
    n_values = [1, 2, 3, 4]
    
    for source_lang in languages:
        for target_lang in languages:
            for n in n_values:
                model = build_lm(source_lang, n, True)
                pp = eval(model, target_lang, n)
                df.loc[len(df)] = [source_lang, target_lang, n, pp]
    
    return df

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [13]:
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  text = ""
  ########################################
  # set seed
  model = build_lm(lang, n, False)
  text = prompt
  np.random.seed(r)
  for i in range(number_of_tokens):
    print("text:", text)
    context = text[-n:]
    print("context:", context)
    options = model.get(context).keys()
    probs = model.get(context).values()
    print("options:", options)
    print("probs:", probs)
    next_char = np.random.choice(options, p=probs)
    text += next_char

  ########################################
  return text

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [14]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

text: I am
context: m


AttributeError: 'NoneType' object has no attribute 'keys'

# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [ ]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }

def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')
########################################

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

total 12
drwxr-xr-x 2 root root 4096 Mar 27 16:54 data
-rw-r--r-- 1 root root  903 Mar 27 17:05 results.json
drwxr-xr-x 1 root root 4096 Mar 24 13:34 sample_data
